In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env", override=True)

import sys
sys.path.append('../')

### ingest


In [2]:
from src.ingest import ingest
ingest(thesis_path="../data/BACHELOR_Andreas.pdf")

### examiner

In [1]:
from langchain_openai import ChatOpenAI
from src.prompts import persona_prompt_with_input
from rich.pretty import pprint

pprint(persona_prompt_with_input)

ModuleNotFoundError: No module named 'src'

In [ ]:
%load_ext autoreload
%autoreload 2
from src.utils import format_messages

user_input = """
uffe just pedersen

Erfaren CFO med bred erfaring indenfor økonomifunktionen; udvikling af bl.a. performance management, økonomiske processer og budgetterings- og planlægningsmodeller samt sikring af korrekt og rettidig rapportering af både regnskabstal og nøgletal.

Opbygning og udvikling af økonomifunktioner i forskellige virksomhedstyper både nationalt og internationalt. International erfaring fra Polen, Spanien, USA, Finland, Irland og Sverige.

Gode resultater med implementering og optimering af administrative IT-systemer.

Ledelsesansvar for op til 25 medarbejdere.

Master degree in management and accounting
"""

language = "English"

prompt = persona_prompt_with_input.invoke({
  "language": language, 
  "user_input": user_input, 
  "human_feedback": "",
  "previous_version": "",
  })
pprint(prompt)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ChatPromptValue(
│   messages=[
│   │   SystemMessage(
│   │   │   content='You are tasked with creating a virtual persona based on the information provided by the user about their examiner.\n\nPlease follow these steps:\n\n1. Review the user_input, which may include details about the examiner\'s education, work experience, skills, and personal interests.\n2. Create a detailed virtual persona that captures the examiner\'s traits, including:\n   - Name\n   - Age\n   - Professional background\n   - Personal interests\n   - Communication style\n3. If human_feedback and previous_version are both provided, use the human feedback to improve the previous version of the persona.\n4. Write the persona in English.\n\n<example>\nUser input: "Dr. Jane Smith, PhD in Psychology, 10 years of experience in clinical practice, passionate about mental health advocacy, enjoys hiking and painting."\nGenerated persona: \n- Name: Dr. Jane Smith\n- Age: 45\n- Professional background: Clinical Psychologist focused on mental health advocacy\n- Personal interests: Hiking, painting\n- Communication style: Empathetic and supportive, often shares personal stories to connect with others.\n\nMake sure the persona is engaging and relatable, and feel free to add unique traits that enhance the character.\n\nBelow is the information you need to follow the guidelines:\n<user_input>\n\nuffe just pedersen\n\nErfaren CFO med bred erfaring indenfor økonomifunktionen; udvikling af bl.a. performance management, økonomiske processer og budgetterings- og planlægningsmodeller samt sikring af korrekt og rettidig rapportering af både regnskabstal og nøgletal.\n\nOpbygning og udvikling af økonomifunktioner i forskellige virksomhedstyper både nationalt og internationalt. International erfaring fra Polen, Spanien, USA, Finland, Irland og Sverige.\n\nGode resultater med implementering og optimering af administrative IT-systemer.\n\nLedelsesansvar for op til 25 medarbejdere.\n\nMaster degree in management and accounting\n\n<user_input/>\n\n<human_feedback>\n\n<human_feedback/>\n\n<previous_version>\n\n<previous_version/>',
│   │   │   additional_kwargs={},
│   │   │   response_metadata={}
│   │   )
│   ]
)

In [ ]:
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.2)
chain = persona_prompt_with_input | model
response = chain.invoke({
  "language": language, 
  "user_input": user_input, 
  "human_feedback": "",
  "previous_version": "",
  })
print(response)

{'name': 'Uffe Just Pedersen', 'age': 50, 'professional': "Experienced CFO with extensive expertise in financial functions, including performance management, financial processes, and budgeting and planning models. Proven track record in ensuring accurate and timely reporting of financial statements and key figures. Skilled in building and developing finance functions across various types of companies both nationally and internationally, with international experience in Poland, Spain, USA, Finland, Ireland, and Sweden. Holds a Master's degree in management and accounting.", 'personal': 'Enjoys exploring new cultures through travel, has a passion for cooking, and is an avid reader of both fiction and non-fiction. Also interested in technology and its impact on business efficiency.', 'communication': 'Direct and analytical, Uffe communicates with clarity and precision. He values open dialogue and encourages team input, often using real-world examples to illustrate complex financial concep

### loader

In [34]:
from langchain_pymupdf4llm import PyMuPDF4LLMLoader

thesis_path = "../data/BACHELOR_Andreas.pdf"
loader = PyMuPDF4LLMLoader(file_path=thesis_path, mode="single")
docs = loader.load()
doc = docs[0]

In [48]:
len(doc.page_content)

92498

In [51]:
doc.metadata["collection_name"] = "data/BACHELOR_Andreas.pdf"

In [52]:
doc.metadata

{'producer': '',
 'creator': 'Microsoft Word',
 'creationdate': '2025-08-11T09:18:24+00:00',
 'source': '../data/BACHELOR_Andreas.pdf',
 'file_path': '../data/BACHELOR_Andreas.pdf',
 'total_pages': 30,
 'format': 'PDF 1.7',
 'title': '',
 'author': 'Microsoft Office-bruger',
 'subject': '',
 'keywords': '',
 'moddate': '2025-08-11T09:18:24+00:00',
 'trapped': '',
 'modDate': "D:20250811091824+00'00'",
 'creationDate': "D:20250811091824+00'00'",
 'collection_name': 'data/BACHELOR_Andreas.pdf'}

In [ ]:
from rich.console import Console
from rich.markdown import Markdown

console = Console()
md = Markdown(doc.page_content)
console.print(md)

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_document = doc.page_content

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers=False)
md_header_splits = markdown_splitter.split_text(markdown_document)
len(md_header_splits)

25

In [28]:
console = Console()
md = Markdown(md_header_splits[19].page_content)
console.print(md)

Kapitel 4: Integreret analyse                                           

I denne del integreres de juridiske og økonomiske perspektiver for at vurdere, hvilken virksomhedsform der samlet  
set er mest fordelagtig for Mikkel under forskellige omstændigheder. Valget er ikke entydigt. Det afhænger af en   
afvejning mellem juridiske og økonomiske aspekter, som i Mikkels case er tæt forbundne. Juridiske faktorer (som    
hæftelsesform, ejerskabsstruktur og formelle krav) påvirker direkte de økonomiske konsekvenser (risikofordeling,   
skattebetaling, finansieringsmuligheder og omkostninger), og omvendt bør Mikkels økonomiske mål og risikoprofil    
styre, hvilken juridisk ramme, der bedst understøtter hans virksomhed. Nedenfor analyseres samspillet, og der      
træffes en samlet vurdering ved hjælp af scenarier og beslutningskriterier fra både juridisk og økonomisk teori.

In [33]:
doc

Document(metadata={'producer': '', 'creator': 'Microsoft Word', 'creationdate': '2025-08-11T09:18:24+00:00', 'source': '../data/BACHELOR_Andreas.pdf', 'file_path': '../data/BACHELOR_Andreas.pdf', 'total_pages': 30, 'format': 'PDF 1.7', 'title': '', 'author': 'Microsoft Office-bruger', 'subject': '', 'keywords': '', 'moddate': '2025-08-11T09:18:24+00:00', 'trapped': '', 'modDate': "D:20250811091824+00'00'", 'creationDate': "D:20250811091824+00'00'"}, page_content='# **Valg af selskabsform – en tværfaglig analyse**\n\nLavet af Andreas la Cour Asboe\n\n\nStudienummer: 161120\n\n\nVejledere: Anders Larsen og Uffe Just Pedersen\n\n\nHA(jur.) CBS\nBachelorprojekt på 6. semester 2025\n\n\nAntal anslag inkl. mellemrum: 90.150\n\nantal sider: 30\n\n\nAfleveret 11/08/2025\n\n\n\n1\n\n\n\n-----\n\n## **_Resume_**\n\n_This thesis examines how a new business can choose between operating as a sole proprietorship (EMV) or a_\n\n\n_private limited company (ApS). The main research question is how an en

In [36]:
# Char-level splits
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 1000
chunk_overlap = 200
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(docs)
len(splits)

135

In [ ]:
from langchain.indexes import SQLRecordManager

recorder = SQLRecordManager(
        namespace=namespace,
        db_url=get_record_db_url(),
    )

In [46]:
console = Console()
md = Markdown(splits[15].page_content)
console.print(md)

17                                                                                                                 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
virksomhed i ApS formentlig være stillet på forholdsvis lige fod med virksomhed i en EMV i relation til muligheden 
for at få ekstern lånefinansiering.                                                                                

Fra Mikkels eget perspektiv er de 20.000 kr. i ApS-kapitalkravet heller ikke tabte midler. De kan som nævnt        
tidligere anvendes i driften og til at købe aktiver for. Indskudt selskabskapital kan ikke udloddes til            
privatforbrug. Udlodning kan tidligst ske efter aflæggelse af første årsrapport og forudsætter i øvrigt, at der er 
overskud i ApS ‘et (som udgør frie midler) jf. SL §180 stk. 2. Kortsigtet kan ApS-formen derfor være mindre likvid 
for Mikkel. Mikkels privatøkonomi og selskabets midler må ikke sammenblandes. Midler optjent i selskabet kan alene 
overgå til Mikkel, hvis der træffes beslutning om udbytteudlodning eller betaling af løn. Derfor bør Mikkel        
foretage nødvendige økonomiske beregninger, så han, efter brug af midler til stiftelse af et ApS (20.000 kr .+     
stiftelsesomkostninger), har tilstrækkelige midler til dækning af sine egne private udgifter.                      

En yderligere sikkerhed for kreditorer i et ApS er, at selskabet skal vedligeholde en forsvarlig egenkapital.      
Derfor indeholder selskabsloven regler for kapitaltab, jf. SL §4 jf. §119. Hvis anpartsselskabets selskabskapital  
falder med mere end halvdelen, dvs. til under 10.000 kr., betragtes det som et kapitaltab, og her er der krav om,  
at ledelsen skal indkalde til generalforsamling senest 6 måneder efter kapitaltabet er konstateret. På             
generalforsamlingen skal ledelsen redegøre for selskabets økonomiske stilling og foreslå tiltag for at afhjælpe    
situationen.                                                                                                       

Mikkel skal i den situation reagere ved at tilføre ny kapital eller afvikle selskabet, hvis egenkapitalen bliver   
for lav jf. SL §4 jf. §119. I økonomiske termer tvinger det ham til en vis finansiel disciplin og foretage en      
løbende vurdering af virksomhedens soliditet, hvilket enten kan betragtes som en byrde eller et positivt           
disciplinært ”push” til at være mere organiseret. I en enkeltmandsvirksomhed har Mikkel mere fleksibilitet til at  
lade midlertidige underskud passere. Han kan fortsætte, så længe han personligt kan dække gælden, uden formelle    
krav om rekapitalisering. Denne fleksibilitet kan være en fordel under midlertidig økonomisk modgang, men den kan  
også medføre, at Mikkel i en EMV kommer til at fortsætte en økonomisk håbløs drift længere end økonomisk rentabelt,
fordi der ikke er et formelt krav om at foretage en reetablering eller lukke virksomheden, hvis der opstår         
kapitaltab.                                                                                                        

Opsummerende kan man sige at nettoeffekten af kapitalkravet afhænger af Mikkels finansielle situation. Har han     
opsparing nok, er 20.000 kr. måske en beskeden pris for de øvrige fordele ved ApS, og hvis han alligevel skal ud og
bruge penge på driftsaktiver til mindst 20.000 kr., er kapitalkravet underordnet, da selskabskapitalen kan benyttes
til anskaffelse af driftsaktiver. Har han derimod begrænset kapital, kunne kravet presse ham til at optage dyr gæld
eller opgive ApS som mulighed. I så fald kan den personlige virksomheds fravær af kapitalkrav være det, der gør det
økonomisk muligt overhovedet at etablere virksomheden. Her er det altså igen vigtigt for Mikkel at vurdere sin     
økonomiske situation, inden han tager et valg.                                                                     

                                                  

### web search

In [17]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5",
    tools=[{
        "type": "web_search_preview",
        "user_location": {
            "type": "approximate",
            "country": "DK",
            "city": "Copenhagen",
            "region": "Denmark",
            },
        "search_context_size": "high",
        }],
    input="Hvilke konkrete kvantitative antagelser lå til grund for din økonomiske analyse (skattesatser, forventet overskud/underskud, kapitalbehov mv.), og hvor følsomme er dine konklusioner over for ændringer i disse antagelser?"
)


In [18]:
from rich.pretty import pprint

pprint(response)

Response(
│   id='resp_68aabeb4cf0c819685ea2c08226bc9ad027168f817c34db1',
│   created_at=1756020404.0,
│   error=None,
│   incomplete_details=None,
│   instructions=None,
│   metadata={},
│   model='gpt-5-2025-08-07',
│   object='response',
│   output=[
│   │   ResponseReasoningItem(
│   │   │   id='rs_68aabeb58b94819687813e2622b955ff027168f817c34db1',
│   │   │   summary=[],
│   │   │   type='reasoning',
│   │   │   content=None,
│   │   │   encrypted_content=None,
│   │   │   status=None
│   │   ),
│   │   ResponseOutputMessage(
│   │   │   id='msg_68aabedb02dc8196b7451465fc4b5121027168f817c34db1',
│   │   │   content=[
│   │   │   │   ResponseOutputText(
│   │   │   │   │   annotations=[],
│   │   │   │   │   text='Godt spørgsmål—men jeg kan ikke se den økonomiske analyse, du henviser til, i denne tråd. Hvis du deler casen eller de base case-tal, jeg brugte (eller du forventer, jeg brugte), kan jeg svare præcist. Indtil da får du både:\n- en standardliste over konkrete kvantitative antagelser, jeg normalt dokumenterer, og\n- hvordan jeg typisk tester følsomheden af konklusionerne.\n\n1) Kvantitative antagelser jeg normalt fastlægger (DK-perspektiv)\n- Tidsramme og valuta: prognosehorisont (fx 5–10 år) og DKK, real vs. nominel.\n- Omsætning: prisniveau, volumen, mix, churn/ARPU (SaaS), sæson, eksportandel, evt. valutakurser.\n- Vækst og inflation: årlig enhedsvækst og prisinflation pr. cost-driver.\n- Brutto- og EBITDA-marginer: COGS/variable omkostninger og faste omkostninger (lønsatser, husleje, energi, S&M, G&A).\n- Investeringer (capex) og afskrivninger: initialt capex, vedligeholdscapex, nyttige levetider, scrapværdi.\n- Nettodriftskapital: DSO, DIO, DPO (dage), evt. forskudsbetalinger.\n- Finansiering: gæld/egenkapital, rente (fast/variabel, margin over reference), afdrag, minimum DSCR, covenant-buffer.\n- Skat og afgifter: selskabsskat (DK pt. 22%), momsbehandling (standard 25% men afhænger af branche/eksport), evt. lønsumsafgift/energiafgifter, skattemæssige afskrivninger og underskudsfremførsel.\n- Diskonteringsrente: WACC (kapitalstruktur, kostnad for egenkapital og gæld, skatteværn).\n- Tilskud/støtte og andre engangsposter.\n\n2) Følsomhed: sådan tester jeg dine konklusioner\n- Enkelt-parameter følsomhed (tornado): ændr én input ad gangen (typisk ±10% eller ±1 procentpoint) og mål effekt på NPV, IRR, payback og DSCR. Ranger effekterne.\n- Scenarier: Base, Upside og Downside (kombinerer fx lavere pris/volumen, højere energi/løn, højere rente).\n- Break-even/thresholds: find grænser hvor NPV = 0, IRR = kapitalkrav, eller DSCR = covenant (fx 1,20x).\n- Regler der ofte gælder:\n  - Pris/volumen er næsten altid største driver, især ved høj driftsmæssig gearing.\n  - WACC: En stigning på 1 procentpoint i WACC reducerer NPV desto mere, jo længere varige cashflows og jo tættere g er på WACC. Brug FCF1/(WACC−g)-intuitionen.\n  - Rente: +200 bp på variabel gæld kan presse DSCR markant; følsomheden afhænger af gældsandelen og afdragstempo.\n  - Bruttomargin: Små ændringer i pct.-point kan flytte NPV meget, hvis faste omkostninger er store.\n  - Nettodriftskapital: En ændring på n dage i DSO ~ (n/365)×årlig omsætning i likviditet (engangseffekt).\n  - Skat: Effektiv skattesats betyder mest, når projektet hurtigt bliver skattemæssigt profitabelt; med underskudsfremførsel udskydes effekten.\n\nVil du have et konkret svar for “din analyse”?\n- Send venligst disse base case-tal (eller bekræft, at jeg må antage standarder nedenfor), så vender jeg tilbage med en kort følsomhedsoversigt:\n  - År 1 omsætning (DKK), årlig vækst (%), pris vs. volumen-drivere\n  - Bruttomargin (%), faste OPEX (DKK) og deres årlige stigning (%)\n  - Capex: initialt (DKK) og som % af omsætning fremadrettet\n  - DSO/DIO/DPO (dage)\n  - Finansiering: gæld/egenkapital, rente og afdrag\n  - WACC (%), inflationsantagelse (%)\n  - Selskabsskat (%), momsforhold\n- Alternativt kan jeg foreslå et “default” DK-setup (fx selskabsskat 22%, moms 25%, WACC 8–10%, vari

In [20]:
print(response.output_text)

Godt spørgsmål—men jeg kan ikke se den økonomiske analyse, du henviser til, i denne tråd. Hvis du deler casen eller de base case-tal, jeg brugte (eller du forventer, jeg brugte), kan jeg svare præcist. Indtil da får du både:
- en standardliste over konkrete kvantitative antagelser, jeg normalt dokumenterer, og
- hvordan jeg typisk tester følsomheden af konklusionerne.

1) Kvantitative antagelser jeg normalt fastlægger (DK-perspektiv)
- Tidsramme og valuta: prognosehorisont (fx 5–10 år) og DKK, real vs. nominel.
- Omsætning: prisniveau, volumen, mix, churn/ARPU (SaaS), sæson, eksportandel, evt. valutakurser.
- Vækst og inflation: årlig enhedsvækst og prisinflation pr. cost-driver.
- Brutto- og EBITDA-marginer: COGS/variable omkostninger og faste omkostninger (lønsatser, husleje, energi, S&M, G&A).
- Investeringer (capex) og afskrivninger: initialt capex, vedligeholdscapex, nyttige levetider, scrapværdi.
- Nettodriftskapital: DSO, DIO, DPO (dage), evt. forskudsbetalinger.
- Finansiering:

### tavily search

In [24]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=2)
search_docs = tavily_search.invoke("Hvilke konkrete kvantitative antagelser lå til grund for din økonomiske analyse (skattesatser, forventet overskud/underskud, kapitalbehov mv.), og hvor følsomme er dine konklusioner over for ændringer i disse antagelser?") 

/var/folders/0g/4n6rzj1n21n6dlmfrdjkpj9m0000gn/T/ipykernel_37894/3668718499.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=2)


In [25]:
pprint(search_docs)

[
│   {
│   │   'title': 'Kap. 3.4 Tallfeste, verdsette og vurdere virkninger (fase 4) - DFØ',
│   │   'url': 'https://dfo.no/fagomrader/utredning-og-analyse-av-statlige-tiltak/samfunnsokonomiske-analyser/veileder-i-samfunnsokonomiske-analyser/kap-34-tallfeste-verdsette-og-vurdere-virkninger-fase-4',
│   │   'content': 'Det er viktig å dokumentere og synliggjøre forutsetningene og faktagrunnlaget bak anslagene og usikkerheten i dem. Er det brukt grove estimater, skal man være åpen om det. Jo større usikkerhet det er knyttet til forutsetningene, desto viktigere blir det å beskrive usikkerheten og gjøre følsomhetsanalyser eller andre typer usikkerhetsanalyser (se kapittel 3.6). [...] ## Dekningsanalyse (break-even-analyse)\n\n## Effektiv ressursbruk\n\n## Effektmål\n\n## Eksterne virkninger\n\n## Enhetsverdi\n\n## Fordelingsvirkninger\n\n## Forenklet analyse\n\n## Forventningsverdi\n\n## Følsomhetsanalyse\n\n## Føre var-prinsippet\n\n## Gevinster\n\n## Ikke prissatte virkninger\n\n## Investeringsprosjekter\n\n## Kalkulasjonspris\n\n## Kalkulasjonsrente\n\n## Kostnadseffektivitetsanalyse\n\n## Kostnadsvirkninger\n\n## Kostnadsvirkningsanalyse\n\n## Kvalitativ vurdering\n\n## Kvantum\n\n## Levetid\n\n## Markedssvikt [...] Det er som regel knyttet usikkerhet til fremtidige tallfestede størrelser og verdsatte virkninger. I en samfunnsøkonomisk analyse beregner man derfor forventningsverdier for nytte- og kostnadsvirkningene til de tiltakene som utredes. Forventningsverdien til en virkning er definert som en veid sum der hvert enkelt mulige utfall for virkningen vektes med den tilhørende sannsynligheten for dette utfallet.',
│   │   'score': 0.25952804
│   },
│   {
│   │   'title': '[EPUB] Samfunnsøkonomiske analyser - Regjeringen.no',
│   │   'url': 'https://www.regjeringen.no/contentassets/5fce956d51364811b8547eebdbcde52c/no/epub/nou201220120016000dddepub.epub',
│   │   'content': 'Utvalgets syn er at samfunnsøkonomisk lønnsomhet skal tolkes som et oppsummerende mål på hva ... konkrete fordelingsvektene og 3) hvilken type prosjekt som',
│   │   'score': 0.057774507
│   }
]

In [28]:
from langchain_core.documents import Document
documents = []
for doc in search_docs:
    metadata = {
        "url": doc["url"],
        "title": doc["title"],
        "type": "web_search"
    }
    documents.append(Document(page_content=doc["content"], metadata=metadata))

In [29]:
type(documents[0])

<class 'langchain_core.documents.base.Document'>